In [7]:
import os
import cv2
import numpy as np
import pandas as pd
from datetime import datetime
from deepface import DeepFace
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pickle
import pandas as pd
from datetime import datetime
import os

# 1. Data Augmentation and Preprocessing
def preprocess_and_augment_images(folder_path, target_size=(224, 224)):
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    
    X, y = [], []
    label_dict = {}
    current_label = 0

    print("Starting data augmentation and preprocessing...")
    for label in os.listdir(folder_path):
        label_path = os.path.join(folder_path, label)
        if os.path.isdir(label_path):
            label_dict[label] = current_label
            current_label += 1
            
            for file_name in os.listdir(label_path):
                file_path = os.path.join(label_path, file_name)
                if file_path.endswith(('.png', '.jpg', '.jpeg')):
                    # Load and preprocess image
                    img = cv2.imread(file_path)
                    if img is None:
                        print(f"Warning: Unable to read image {file_path}")
                        continue
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, target_size)
                    img = img / 255.0  # Normalize

                    # Apply augmentation
                    img = np.expand_dims(img, axis=0)
                    for augmented in datagen.flow(img, batch_size=1):
                        X.append(augmented[0])
                        y.append(label_dict[label])
                        break  # Generate one augmented image per original

    return np.array(X), np.array(y), label_dict

# 2. Extract Face Embeddings
def extract_face_embeddings(images, labels):
    embeddings, new_labels = [], []

    print("Extracting face embeddings...")
    for img, label in zip(images, labels):
        try:
            # Convert image to uint8 format for DeepFace
            img_uint8 = (img * 255).astype(np.uint8)
            embedding = DeepFace.represent(img_path=img_uint8, model_name='Facenet')[0]['embedding']
            embeddings.append(embedding)
            new_labels.append(label)
        except Exception as e:
            print(f"Error processing image for label {label}: {e}")

    return np.array(embeddings), np.array(new_labels)

# 3. Train SVM Model
def train_svm(X, y):
    print("Training SVM model...")
    svm_model = SVC(kernel='linear', probability=True)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    svm_model.fit(X_train, y_train)

    # Evaluate
    predictions = svm_model.predict(X_test)
    print("Classification Report:")
    print(classification_report(y_test, predictions))
    print(f"Accuracy: {accuracy_score(y_test, predictions) * 100:.2f}%")

    return svm_model

# 4. Real-Time Attendance
def real_time_attendance(model, label_dict):
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open webcam")
        return

    print("Starting real-time attendance system...")
    print("Press 'q' to quit")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Could not read frame")
            break

        try:
            # Extract face embedding using DeepFace
            result = DeepFace.represent(frame, model_name='Facenet', enforce_detection=False)
            if result:
                face_embedding = result[0]["embedding"]
                face_embedding = np.array(face_embedding).reshape(1, -1)

                # Predict using SVM model
                prediction = model.predict(face_embedding)[0]
                confidence = model.predict_proba(face_embedding).max()

                # Get name from label
                name = list(label_dict.keys())[list(label_dict.values()).index(prediction)]

                # Get face location from DeepFace
                face_objs = DeepFace.extract_faces(frame, enforce_detection=False)
                if face_objs:
                    for face_obj in face_objs:
                        facial_area = face_obj['facial_area']
                        x = facial_area['x']
                        y = facial_area['y']
                        w = facial_area['w']
                        h = facial_area['h']

                        # Draw rectangle and name with confidence
                        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                        display_text = f"{name} ({confidence:.2f})"
                        cv2.putText(frame, display_text, (x, y - 10), 
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

                        # Mark attendance if confidence is high enough
                        if confidence > 0.6:  # Adjust threshold as needed
                            mark_attendance(name)
        except Exception as e:
            print(f"Error during face recognition: {e}")

        cv2.imshow('Real-Time Attendance', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

#This will create an Excel File of Students present im Class today 
def mark_attendance(name):
    now = datetime.now()
    date = now.strftime("%Y-%m-%d")
    time = now.strftime("%H:%M:%S")

    attendance_file = 'attendance.xlsx'

    try:
        # Check if file exists
        if os.path.exists(attendance_file):
            # Read existing Excel file
            df = pd.read_csv(attendance_file)
        else:
            # Create new DataFrame if file doesn't exist
            df = pd.DataFrame(columns=['Name', 'Date', 'Time'])

        # Check for duplicate entries for same name and date
        existing_entry = df[(df['Name'] == name) & (df['Date'] == date)]
        if existing_entry.empty:
            # Append new attendance record
            new_record = pd.DataFrame({
                'Name': [name], 
                'Date': [date], 
                'Time': [time]
            })
            df = pd.concat([df, new_record], ignore_index=True)
            
            # Save to Excel
            df.to_excel(attendance_file, index=False)
            print(f"Attendance marked for {name}")
        else:
            print(f"{name} already marked present today")

    except Exception as e:
        print(f"Error marking attendance: {e}")

def main():
    folder_path = r"F:\\Facial Dataset"

    try:
        # Step 1: Preprocess and augment images
        print("\nInitializing face recognition system...")
        X, y, label_dict = preprocess_and_augment_images(folder_path)

        if len(X) == 0:
            print("Error: No images processed!")
            return

        # Step 2: Extract face embeddings
        embeddings, labels = extract_face_embeddings(X, y)

        # Step 3: Train SVM model
        svm_model = train_svm(embeddings, labels)

        # Save the model and label dictionary
        with open("svm_model.pkl", "wb") as f:
            pickle.dump(svm_model, f)
        with open("label_dict.pkl", "wb") as f:
            pickle.dump(label_dict, f)

        # Step 4: Real-time attendance
        real_time_attendance(svm_model, label_dict)

    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()


Initializing face recognition system...
Starting data augmentation and preprocessing...
Extracting face embeddings...
Error processing image for label 0: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error processing image for label 0: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error processing image for label 0: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error processing image for label 0: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error processing image for label 2: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection pa